## Business Problem section 

#### Background

House prices in many parts of Coventry have been falling for some time as the overpriced market corrects itself, but new research suggests the long slump could finally be coming to an end.The Daily Telegraph’s Marianna Hunt says “dampened demand, as buyers were put off by tax changes, high prices and poor value for money”, combined with uncertainty around Brexit has seen the number of sales in the capital fall by a quarter over the past five years.

Now new research by property website Zoopla suggests the slump in house price growth may be slowing. In October 2018 the level of house price falls plateaued and the number of areas in Coventry registering a drop in prices has since fallen from 80% in October to 68% today. Reference -https://www.theweek.co.uk/london-house-prices

Business Problem
We create a ML model to cluster the housing in the Coventry region based on price, nearby facilities and guid the investors and homebuyers to make the right choice.

#### Data section

HM Land Registry publishes the following public datasets on GOV.UK as part of  the Government’s priorities of economic growth and data transparency:Price paid data updated monthly, data available from 1995.Transaction data updated monthly, data available from December 2011.UK House Price Index downloads updated monthly, data available from January 1995. HM Land Registry publish the UK House Price Index on behalf of Office for National Statistics, Registers of Scotland and Land and Property Services Northern Ireland. These datasets are provided in comma-separated value (csv) and linked data formats, with Price Paid Data also available as a text file.(http://landregistry.data.gov.uk/).


To explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we will access data through FourSquare API interface and arrange them as a dataframe for visualization. By merging data on Coventry properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we will be able to recommend profitable real estate investments.

#### Methodology section

The Methodology section will describe the main components of our analysis and predication system. The Methodology section comprises four stages:

1. Collect Inspection Data
2. Explore and Understand Data
3. Data preparation and preprocessing 
4. Modeling

In [1]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


import folium #import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [ ]:
#Read the data for examination (Source: http://landregistry.data.gov.uk/)
#df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

In [21]:
df_ppd =pd.read_csv(r'C:\Users\134318\Dropbox\Coursera\DS Pro Certification\Capstone\pp-2018.csv')

In [22]:
df_ppd.head()

,{7E86B6FA-70B7-458C-E053-6B04A8C0C84C},435000,2018-06-22 00:00,UB10 8FQ,F,Y,L,HINDS COURT,FLAT 19,PERKINS GARDENS,ICKENHAM,UXBRIDGE,HILLINGDON,GREATER LONDON,A,A.1
0,{7E86B6FA-70B8-458C-E053-6B04A8C0C84C},355000,2018-10-19 00:00,UB3 1DZ,F,Y,L,"BOILER HOUSE, 2",FLAT 54,MATERIAL WALK,NaN,HAYES,HILLINGDON,GREATER LONDON,A,A
1,{7E86B6FA-70B9-458C-E053-6B04A8C0C84C},465000,2018-09-14 00:00,EN5 2FQ,F,Y,L,"DELPHI HOUSE, 4",FLAT 5,HERA AVENUE,NaN,BARNET,BARNET,GREATER LONDON,A,A
2,{7E86B6FA-70BA-458C-E053-6B04A8C0C84C},540000,2018-09-14 00:00,EN5 2FQ,F,Y,L,"DELPHI HOUSE, 4",FLAT 17,HERA AVENUE,NaN,BARNET,BARNET,GREATER LONDON,A,A
3,{7E86B6FA-70BB-458C-E053-6B04A8C0C84C},415000,2018-10-02 00:00,N13 5EX,F,Y,L,"HAZELTREE LODGE, 16 - 18",FLAT 9,HAZELWOOD LANE,NaN,LONDON,ENFIELD,GREATER LONDON,A,A
4,{7E86B6FA-70BC-458C-E053-6B04A8C0C84C},470000,2018-09-17 00:00,EN5 2FQ,F,Y,L,"DELPHI HOUSE, 4",FLAT 21,HERA AVENUE,NaN,BARNET,BARNET,GREATER LONDON,A,A


In [23]:
# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [24]:
df_ppd.head()

,TUID,Price,Date_Transfer,Postcode,Prop_Type,Old_New,Duration,PAON,SAON,Street,Locality,Town_City,District,County,PPD_Cat_Type,Record_Status
0,{7E86B6FA-70B8-458C-E053-6B04A8C0C84C},355000,2018-10-19 00:00,UB3 1DZ,F,Y,L,"BOILER HOUSE, 2",FLAT 54,MATERIAL WALK,NaN,HAYES,HILLINGDON,GREATER LONDON,A,A
1,{7E86B6FA-70B9-458C-E053-6B04A8C0C84C},465000,2018-09-14 00:00,EN5 2FQ,F,Y,L,"DELPHI HOUSE, 4",FLAT 5,HERA AVENUE,NaN,BARNET,BARNET,GREATER LONDON,A,A
2,{7E86B6FA-70BA-458C-E053-6B04A8C0C84C},540000,2018-09-14 00:00,EN5 2FQ,F,Y,L,"DELPHI HOUSE, 4",FLAT 17,HERA AVENUE,NaN,BARNET,BARNET,GREATER LONDON,A,A
3,{7E86B6FA-70BB-458C-E053-6B04A8C0C84C},415000,2018-10-02 00:00,N13 5EX,F,Y,L,"HAZELTREE LODGE, 16 - 18",FLAT 9,HAZELWOOD LANE,NaN,LONDON,ENFIELD,GREATER LONDON,A,A
4,{7E86B6FA-70BC-458C-E053-6B04A8C0C84C},470000,2018-09-17 00:00,EN5 2FQ,F,Y,L,"DELPHI HOUSE, 4",FLAT 21,HERA AVENUE,NaN,BARNET,BARNET,GREATER LONDON,A,A


In [25]:
df_ppd.shape

(1011413, 16)

In [26]:
df_ppd['Date_Transfer']=df_ppd['Date_Transfer'].apply(pd.to_datetime)

In [27]:
dropyears=df_ppd.Date_Transfer.dt.year<2016
dropyears.shape


(1011413L,)

In [28]:
df_ppd[dropyears].index



Int64Index([], dtype='int64')

In [29]:
df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index

Int64Index([], dtype='int64')

In [30]:
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [31]:
df_ppd.head()

,TUID,Price,Date_Transfer,Postcode,Prop_Type,Old_New,Duration,PAON,SAON,Street,Locality,Town_City,District,County,PPD_Cat_Type,Record_Status
237810,{8355F009-6070-55C5-E053-6B04A8C0D090},370000,2018-12-31,SE25 6TX,T,N,F,13,NaN,BROSTER GARDENS,NaN,LONDON,CROYDON,GREATER LONDON,A,A
191732,{87E1551E-F60B-6405-E053-6C04A8C0B2EE},100000,2018-12-31,CT6 5QA,O,N,L,155,NaN,STATION ROAD,NaN,HERNE BAY,CANTERBURY,KENT,B,A
480842,{80E1AA97-E35D-7BF8-E053-6C04A8C00BF2},190000,2018-12-31,PL1 4EJ,T,N,F,129,NaN,KER STREET,NaN,PLYMOUTH,CITY OF PLYMOUTH,CITY OF PLYMOUTH,A,A
21233,{80E1AA99-28B3-7BF8-E053-6C04A8C00BF2},195000,2018-12-31,LS27 8YP,S,N,L,11,NaN,LEYBURN AVENUE,MORLEY,LEEDS,LEEDS,WEST YORKSHIRE,B,A
214718,{8355F008-D75F-55C5-E053-6B04A8C0D090},227500,2018-12-31,PL9 8FX,T,Y,F,36,NaN,DORADO STREET,SHERFORD,PLYMOUTH,SOUTH HAMS,DEVON,A,A


In [32]:
df_ppd['Town_City']

237810                 LONDON
191732              HERNE BAY
480842               PLYMOUTH
21233                   LEEDS
214718               PLYMOUTH
975496                REDHILL
217098         SOUTH OCKENDON
221902                 LONDON
970413        CHIPPING NORTON
249820                  WIGAN
250598                 LONDON
130912              BLACKPOOL
199072               PLYMOUTH
977489              LIVERPOOL
19280                   LEEDS
13377              BIRMINGHAM
993352             MANCHESTER
180385                BRISTOL
989139                 HARROW
15287                 SWANSEA
35115                   TRURO
32306      WELWYN GARDEN CITY
1005006           SOUTHAMPTON
52542                   LEEDS
999224              SHEERNESS
29100                   LEEDS
254476              SHEFFIELD
226700           MARKET RASEN
254477              SHEFFIELD
188732                 OXFORD
                  ...        
685639                   BURY
130547                BRISTOL
1010270   

In [43]:
df_ppd_coventry = df_ppd.query("Town_City == 'COVENTRY'")

# Make a list of street names in LONDON
streets = df_ppd_coventry['Street'].unique().tolist()

In [44]:
sorted(streets)


[nan,
 'ABBEY COTTAGES',
 'ABBEY COURT',
 'ABBEY ROAD',
 'ABBEYDALE CLOSE',
 'ABBOTSBURY CLOSE',
 'ABBOTTS WALK',
 'ABERCORN ROAD',
 'ABERDEEN CLOSE',
 'ABERGAVENNY WALK',
 'ADARE DRIVE',
 'ADDENBROOKE ROAD',
 'ADDISON ROAD',
 'ADELAIDE STREET',
 'AGINCOURT ROAD',
 'AINSBURY ROAD',
 'AINSDALE CLOSE',
 'ALBANY ROAD',
 'ALBERT CRESCENT',
 'ALBERT ROAD',
 'ALDBOURNE ROAD',
 'ALDBURY RISE',
 'ALDER LANE',
 'ALDER MEADOW CLOSE',
 'ALDER ROAD',
 'ALDERMANS GREEN ROAD',
 'ALDERMINSTER ROAD',
 'ALDERMOOR LANE',
 'ALDERNEY CLOSE',
 'ALDRICH AVENUE',
 'ALDRIN WAY',
 'ALEX GRIERSON CLOSE',
 'ALFALL ROAD',
 'ALFRISTON ROAD',
 'ALGATE CLOSE',
 'ALISON SQUARE',
 'ALLAN ROAD',
 'ALLESLEY HALL DRIVE',
 'ALLESLEY OLD ROAD',
 'ALLIANCE WAY',
 'ALPINE RISE',
 'ALSPATH ROAD',
 'ALUM CLOSE',
 'ALVERLEY ROAD',
 'AMBLER GROVE',
 'AMBLESIDE',
 'AMELIA CRESCENT',
 'AMERSHAM CLOSE',
 'AMY CLOSE',
 'ANCHORWAY ROAD',
 'ANDERTON ROAD',
 'ANGELA AVENUE',
 'ANGLIAN WAY',
 'ANGUS CLOSE',
 'ANSELL DRIVE',
 'ANSON WAY'

In [45]:
df_ppd_coventry.head()

,TUID,Price,Date_Transfer,Postcode,Prop_Type,Old_New,Duration,PAON,SAON,Street,Locality,Town_City,District,County,PPD_Cat_Type,Record_Status
158318,{85866A64-FAC6-143F-E053-6B04A8C06A15},178995,2018-12-26,CV6 5LD,T,Y,F,20,NaN,PARAGON WAY,NaN,COVENTRY,COVENTRY,WEST MIDLANDS,A,A
30710,{80E1AA99-219C-7BF8-E053-6C04A8C00BF2},73195,2018-12-21,CV3 4DQ,F,N,L,12,NaN,SUNNYBANK AVENUE,NaN,COVENTRY,COVENTRY,WEST MIDLANDS,B,A
26750,{80E1AA99-213F-7BF8-E053-6C04A8C00BF2},285000,2018-12-21,CV1 4AR,T,N,F,7,NaN,COUNDON ROAD,NaN,COVENTRY,COVENTRY,WEST MIDLANDS,B,A
9442,{80E1AA99-2266-7BF8-E053-6C04A8C00BF2},73195,2018-12-21,CV3 4DZ,F,N,L,140,NaN,SEDGEMOOR ROAD,NaN,COVENTRY,COVENTRY,WEST MIDLANDS,B,A
9436,{80E1AA99-225D-7BF8-E053-6C04A8C00BF2},155000,2018-12-21,CV1 2AW,T,N,F,30,NaN,TERRY ROAD,NaN,COVENTRY,COVENTRY,WEST MIDLANDS,B,A


In [47]:
df_grp_price = df_ppd_coventry.groupby(['Street'])['Price'].mean().reset_index()
df_grp_price.columns = ['Street', 'Avg_Price']

In [48]:
df_grp_price.head(20)

,Street,Avg_Price
0,ABBEY COTTAGES,137125.000000
1,ABBEY COURT,97250.000000
2,ABBEY ROAD,207833.333333
3,ABBEYDALE CLOSE,186400.000000
4,ABBOTSBURY CLOSE,162000.000000
5,ABBOTTS WALK,183750.000000
6,ABERCORN ROAD,215725.000000
7,ABERDEEN CLOSE,290000.000000
8,ABERGAVENNY WALK,260000.000000
9,ADARE DRIVE,158750.000000


In [49]:
#Input your Budget's Upper Limit and Lower Limit - Find the locations df_grp_price which fits your budget
df_affordable = df_grp_price.query("(Avg_Price >= 500000) & (Avg_Price <= 1000000)")

In [50]:
df_affordable.head()

,Street,Avg_Price
21,ALDER LANE,925000.0
68,ARMORIAL ROAD,530000.0
74,ASBURY ROAD,520000.0
90,ATHERON GROVE,500000.0
101,BACK LANE,775000.0


In [51]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [62]:
for index, item in df_affordable.iterrows():
    print  "index :",index
    print  "item  :",item
    print "Street  :" ,item.Street

index : 21
item  : Street       ALDER LANE
Avg_Price        925000
Name: 21, dtype: object
Street  : ALDER LANE
index : 68
item  : Street       ARMORIAL ROAD
Avg_Price           530000
Name: 68, dtype: object
Street  : ARMORIAL ROAD
index : 74
item  : Street       ASBURY ROAD
Avg_Price         520000
Name: 74, dtype: object
Street  : ASBURY ROAD
index : 90
item  : Street       ATHERON GROVE
Avg_Price           500000
Name: 90, dtype: object
Street  : ATHERON GROVE
index : 101
item  : Street       BACK LANE
Avg_Price       775000
Name: 101, dtype: object
Street  : BACK LANE
index : 109
item  : Street       BALSALL STREET EAST
Avg_Price                 850000
Name: 109, dtype: object
Street  : BALSALL STREET EAST
index : 120
item  : Street       BARRETTS LANE
Avg_Price           573750
Name: 120, dtype: object
Street  : BARRETTS LANE
index : 129
item  : Street       BATES ROAD
Avg_Price        715000
Name: 129, dtype: object
Street  : BATES ROAD
index : 135
item  : Street       BAYTON RO

In [67]:
geolocator = Nominatim()

D:\Anaconda2-64\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


In [69]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

D:\Anaconda2-64\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [70]:
df_affordable

,Street,Avg_Price,city_coord
21,ALDER LANE,925000.000000,"(34.15582155, -117.464763677)"
68,ARMORIAL ROAD,530000.000000,"(52.3901488, -1.5190568)"
74,ASBURY ROAD,520000.000000,"(52.3852821, -1.6518428)"
90,ATHERON GROVE,500000.000000,"(40.8528149, 25.877417)"
101,BACK LANE,775000.000000,"(51.9548168, -1.9710412)"
109,BALSALL STREET EAST,850000.000000,"(52.3853785, -1.6549582)"
120,BARRETTS LANE,573750.000000,"(52.1543726, 1.0461715)"
129,BATES ROAD,715000.000000,"(32.316045, -83.301599)"
135,BAYTON ROAD,682500.000000,"(52.46825, -1.4711611)"
152,BEECHWOOD AVENUE,560812.500000,"(54.85552785, -6.29058069012)"


In [71]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

D:\Anaconda2-64\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [72]:
df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
21,ALDER LANE,925000.000000,"(34.15582155, -117.464763677)",34.155822,-117.464764
68,ARMORIAL ROAD,530000.000000,"(52.3901488, -1.5190568)",52.390149,-1.519057
74,ASBURY ROAD,520000.000000,"(52.3852821, -1.6518428)",52.385282,-1.651843
90,ATHERON GROVE,500000.000000,"(40.8528149, 25.877417)",40.852815,25.877417
101,BACK LANE,775000.000000,"(51.9548168, -1.9710412)",51.954817,-1.971041
109,BALSALL STREET EAST,850000.000000,"(52.3853785, -1.6549582)",52.385379,-1.654958
120,BARRETTS LANE,573750.000000,"(52.1543726, 1.0461715)",52.154373,1.046172
129,BATES ROAD,715000.000000,"(32.316045, -83.301599)",32.316045,-83.301599
135,BAYTON ROAD,682500.000000,"(52.46825, -1.4711611)",52.468250,-1.471161
152,BEECHWOOD AVENUE,560812.500000,"(54.85552785, -6.29058069012)",54.855528,-6.290581


In [75]:
address = 'Coventry, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Coventry City are {}, {}.'.format(latitude, longitude))

D:\Anaconda2-64\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Coventry City are 52.4081812, -1.510477.


In [78]:
# create map of Coventry using latitude and longitude values
map_coventry = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df_affordable['Latitude'], df_affordable['Longitude'], df_affordable['Avg_Price'], df_affordable['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_coventry)  
    
map_coventryCo

In [79]:
CLIENT_ID = 'PHEVVSTCZ4PB1ZKX005QY4WE2NRXEDNI4AB5VZDGFGW4JGVX' # your Foursquare ID
CLIENT_SECRET = 'LOU3Y2IQV5DVIJZMXM0JVBF1W1WWQDVMEJWNWI4T34V1GW1Q' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PHEVVSTCZ4PB1ZKX005QY4WE2NRXEDNI4AB5VZDGFGW4JGVX
CLIENT_SECRET:LOU3Y2IQV5DVIJZMXM0JVBF1W1WWQDVMEJWNWI4T34V1GW1Q


We will analyse every neighbourhood with   amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.



#### 4  Modeling
We will use K-means clustering algorithm for gain inshights into neighbourhoods around affordable locations. 

In [80]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [81]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df_affordable['Street'],
                                   latitudes=df_affordable['Latitude'],
                                   longitudes=df_affordable['Longitude']
                                  )

ALDER LANE
ARMORIAL ROAD
ASBURY ROAD
ATHERON GROVE
BACK LANE
BALSALL STREET EAST
BARRETTS LANE
BATES ROAD
BAYTON ROAD
BEECHWOOD AVENUE
BELVEDERE ROAD
BERKELEY ROAD SOUTH
BRACEBRIDGE CLOSE
BRANSFORD AVENUE
BROADWAY MANSIONS
BROADWELLS CRESCENT
CASSANDRA CLOSE
COLLIERY LANE
COMMON LANE
CRYFIELD GRANGE ROAD
CRYFIELD HEIGHTS
DALE MEADOW CLOSE
DUNCHURCH CLOSE
FAIRFAX STREET
FAIRLANDS PARK
FARTHING WALK
FLOYD GROVE
FOLLIS WALK
GALMINGTON DRIVE
GREEN END ROAD
GREENFIELD AVENUE
GREYFRIARS LANE
GUINEA CRESCENT
HAMPTON GRANGE
HARVEST HILL LANE
HAWTHORN DRIVE
HEATH GREEN WAY
HOB LANE
IVY FARM LANE
JACOB DRIVE
KELSEY LANE
KINWALSEY LANE
LETITIA AVENUE
LEYMERE CLOSE
LODGE GREEN LANE
LONDON ROAD
MEETING HOUSE LANE
MERIDEN ROAD
MOREALL MEADOWS
OLD MILL AVENUE
OLD WASTE LANE
PARKSIDE
PICKFORD GRANGE LANE
PICKFORD GREEN LANE
PRIORY ROAD
REGENCY DRIVE
SANDRINGHAM CLOSE
SARACEN DRIVE
SHORTFIELD CLOSE
SLOWLEY HILL
SOUTHLEIGH AVENUE
SPEEDWELL DRIVE
SPENCERS LANE
ST MARTINS ROAD
ST MARYS ROAD
STIVICHALL CRO

In [82]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALDER LANE,34.155822,-117.464764,Fontana Park Aquatic Center,34.152043,-117.463533,Pool
1,ALDER LANE,34.155822,-117.464764,Fontana Bark Park,34.153716,-117.462966,Dog Run
2,ALDER LANE,34.155822,-117.464764,Action Park Alliance Skatepark,34.153084,-117.465942,Skate Park
3,ALDER LANE,34.155822,-117.464764,Fontana Park Roller Hockey Rink,34.152669,-117.463946,Athletics & Sports
4,ARMORIAL ROAD,52.390149,-1.519057,War Memorial Park,52.390205,-1.526246,Park
5,ARMORIAL ROAD,52.390149,-1.519057,Open Arms,52.391794,-1.514171,Pub
6,ARMORIAL ROAD,52.390149,-1.519057,The Beech Tree Cafe,52.391260,-1.523671,Café
7,ARMORIAL ROAD,52.390149,-1.519057,Visitor Centre,52.391360,-1.523716,Park
8,ASBURY ROAD,52.385282,-1.651843,The White Horse,52.387584,-1.646942,Pub
9,ASBURY ROAD,52.385282,-1.651843,berkswell cricket club,52.385894,-1.646094,Athletics & Sports


In [83]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 174 uniques categories.


In [84]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALDER LANE,4,4,4,4,4,4
ARMORIAL ROAD,4,4,4,4,4,4
ASBURY ROAD,3,3,3,3,3,3
ATHERON GROVE,18,18,18,18,18,18
BACK LANE,7,7,7,7,7,7
BARRETTS LANE,4,4,4,4,4,4
BAYTON ROAD,3,3,3,3,3,3
BEECHWOOD AVENUE,2,2,2,2,2,2
BELVEDERE ROAD,10,10,10,10,10,10


In [85]:
location_venues.shape

(592, 7)

In [86]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,Accessories Store,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,...,Train Station,Vacation Rental,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,ALDER LANE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALDER LANE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALDER LANE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALDER LANE,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ARMORIAL ROAD,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
coventry_grouped = venues_onehot.groupby('Street').mean().reset_index()
coventry_grouped

,Street,Accessories Store,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,...,Train Station,Vacation Rental,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,ALDER LANE,0.000000,0.000000,0.000000,0.000000,0.0000,0.250000,0.00,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,ARMORIAL ROAD,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,ASBURY ROAD,0.000000,0.000000,0.000000,0.000000,0.0000,0.333333,0.00,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,ATHERON GROVE,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.055556,0.000000,0.000000,0.055556,0.000000
4,BACK LANE,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,BARRETTS LANE,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,BAYTON ROAD,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,BEECHWOOD AVENUE,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,BELVEDERE ROAD,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,BERKELEY ROAD SOUTH,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000,0.00,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [89]:
num_top_venues = 5

for hood in coventry_grouped['Street']:
    print("----"+hood+"----")
    temp = coventry_grouped[coventry_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALDER LANE----
                   venue  freq
0                   Pool  0.25
1                Dog Run  0.25
2     Athletics & Sports  0.25
3             Skate Park  0.25
4  Performing Arts Venue  0.00


----ARMORIAL ROAD----
               venue  freq
0               Park  0.50
1                Pub  0.25
2               Café  0.25
3  Accessories Store  0.00
4          Newsstand  0.00


----ASBURY ROAD----
                venue  freq
0                 Pub  0.33
1  Athletics & Sports  0.33
2      Massage Studio  0.33
3  Persian Restaurant  0.00
4           Newsstand  0.00


----ATHERON GROVE----
          venue  freq
0          Park  0.11
1        Bakery  0.11
2          Café  0.06
3  Dance Studio  0.06
4   Coffee Shop  0.06


----BACK LANE----
           venue  freq
0            Pub  0.57
1  Grocery Store  0.14
2           Café  0.14
3     Restaurant  0.14
4      Newsstand  0.00


----BARRETTS LANE----
            venue  freq
0             Pub  0.25
1  Sandwich Place  0.25
2    Socc

                    venue  freq
0            Home Service  0.50
1             Auto Garage  0.25
2  Furniture / Home Store  0.25
3       Accessories Store  0.00
4   Performing Arts Venue  0.00


----SANDRINGHAM CLOSE----
                   venue  freq
0            Supermarket  0.33
1      Convenience Store  0.33
2     Chinese Restaurant  0.33
3      Accessories Store  0.00
4  Performing Arts Venue  0.00


----SHORTFIELD CLOSE----
                venue  freq
0   Indian Restaurant  0.43
1               Hotel  0.29
2       Grocery Store  0.14
3  Chinese Restaurant  0.14
4           Nightclub  0.00


----SPEEDWELL DRIVE----
                        venue  freq
0                 Gas Station  0.33
1                   Gastropub  0.33
2  Construction & Landscaping  0.33
3           Accessories Store  0.00
4       Performing Arts Venue  0.00


----ST MARTINS ROAD----
               venue  freq
0                Pub   0.1
1        Supermarket   0.1
2         Restaurant   0.1
3        Pizza Place   

In [90]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [91]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


In [93]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = coventry_grouped['Street']

for ind in np.arange(coventry_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(coventry_grouped.iloc[ind, :], num_top_venues)

In [94]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALDER LANE,Pool,Dog Run,Athletics & Sports,Skate Park,Yoga Studio,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop
1,ARMORIAL ROAD,Park,Pub,Café,Event Space,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fast Food Restaurant
2,ASBURY ROAD,Pub,Athletics & Sports,Massage Studio,Yoga Studio,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
3,ATHERON GROVE,Bakery,Park,Piano Bar,Food Court,Shoe Store,Men's Store,Bookstore,Stadium,Supermarket,Coffee Shop
4,BACK LANE,Pub,Café,Restaurant,Grocery Store,Yoga Studio,Event Space,Food & Drink Shop,Food,Flower Shop,Flea Market


In [97]:
df = df_affordable.drop(columns=['city_coord'])
coventry_grouped=df

In [98]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

coventry_grouped_clustering = coventry_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(coventry_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([2, 4, 4, 4, 0, 2, 1, 3, 3, 1, 3, 4, 1, 4, 0, 4, 1, 0, 1, 2, 0, 1,
       4, 0, 3, 1, 4, 1, 4, 3, 4, 1, 4, 2, 3, 3, 1, 3, 3, 4, 4, 3, 4, 3,
       0, 2, 3, 3, 1, 1])

In [99]:
coventry_grouped_clustering.head()

,Avg_Price,Latitude,Longitude
21,925000.0,34.155822,-117.464764
68,530000.0,52.390149,-1.519057
74,520000.0,52.385282,-1.651843
90,500000.0,40.852815,25.877417
101,775000.0,51.954817,-1.971041


In [122]:
#Dataframe to include Clusters

coventry_grouped_clustering=df.copy()
coventry_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude,Cluster Labels
21,ALDER LANE,925000.0,34.155822,-117.464764,2
68,ARMORIAL ROAD,530000.0,52.390149,-1.519057,4
74,ASBURY ROAD,520000.0,52.385282,-1.651843,4
90,ATHERON GROVE,500000.0,40.852815,25.877417,4
101,BACK LANE,775000.0,51.954817,-1.971041,0


In [123]:
# add clustering labels
coventry_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge Covnetry group with covnetry data to add latitude/longitude for each neighborhood
coventry_grouped_clustering = coventry_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

coventry_grouped_clustering.head(30) # check the last columns!
finaldf=coventry_grouped_clustering;

In [127]:


coventryZip=zip(finaldf['Latitude'], finaldf['Longitude'], finaldf['Street'], finaldf['Cluster Labels'])



In [129]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in coventryZip:
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Results and Discussion section


We may discuss our results under two main perspectives.

First, we may examine them according to neighborhoods/Coventry areas. It is interesting to note that,  SW Coventry and North-West Coventry  might be considered rightvenues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, and affordability. 

Second, we may analyze our results according to the five clusters we have produced. Even though, all clusters could praise an optimal range of facilities and amenities, we have found two main patterns. The first pattern we are referring to, i.e. Clusters 0, 2 and 4, may target home buyers prone to live in 'green' areas with parks, waterfronts. Instead, the second pattern we are referring to, i.e. Clusters 1 and 3, may target individuals who love pubs, theatres and soccer.



#### Conclusion


First, we gathered data on Coventry properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). Moreover, to explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we accessed data through FourSquare API interface and arranged them as a data frame for visualization. By merging data on 
properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we were able to recommend profitable real estate investments. 

Second, The Methodology section comprised four stages: 1. Collect Inspection Data; 2. Explore and Understand Data; 3. Data preparation and preprocessing; 4. Modeling. In particular, in the modeling section, we used the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in Coventry and is accurate.

Finally, we showed the clusters catagries that use would be interested in purchase of affordable property with relevant amenities
